### Open File

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
from matplotlib import pyplot as plt

In [2]:
df_subset = pickle.load(open('./sav/df_subset.sav', 'rb'))

In [3]:
y = df_subset['Category']
X = df_subset[df_subset.columns[1:-1]]

Split into Train and Test

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [5]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((2000, 38), (2000,), (8000, 38), (8000,))

## MODELS

### Functions

In [6]:
import time
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, precision_score, recall_score, make_scorer
from collections import Counter
from sklearn.neighbors import KDTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials, space_eval
from catboost import CatBoostClassifier
import xgboost as xgb

In [7]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    clf = model
    clf.fit(X_train, y_train)
    print_result(clf, X_train, X_test, y_train, y_test)
    return(clf.predict(X_test))

In [9]:
def print_result(clf, X_train, X_test, y_train, y_test):
    print('Accuracy Test :', f'{accuracy_score(y_test, clf.predict(X_test)):.4f}', 
          '| F1 Test :', f'{f1_score(y_test, clf.predict(X_test), pos_label="Bad"):.4f}',
          '| Precision Test :', f'{precision_score(y_test, clf.predict(X_test), pos_label="Bad"):.4f}', 
          '| Recall Test :', f'{recall_score(y_test, clf.predict(X_test), pos_label="Bad"):.4f}', 
          '| H Test :', f'{H_score(y_test, clf.predict(X_test)):.4f}')
    
    print('Accuracy Test :', f'{accuracy_score(y_train, clf.predict(X_train)):.4f}', 
          '| F1 Test :', f'{f1_score(y_train, clf.predict(X_train), pos_label="Bad"):.4f}',
          '| Precision Test :', f'{precision_score(y_train, clf.predict(X_train), pos_label="Bad"):.4f}', 
          '| Recall Test :', f'{recall_score(y_train, clf.predict(X_train), pos_label="Bad"):.4f}', 
          '| H Test :', f'{H_score(y_train, clf.predict(X_train)):.4f}')

In [10]:
def H_score(X_train, y_train):
    acc = accuracy_score(y_train, X_train)
    f1 = f1_score(y_train, X_train, pos_label = "Bad")
    return(2 / ((1/(acc+0.0000001))+(1/(f1+0.0000001))))

In [11]:
def bayesian(space, X, y, modelo, nevals):
    
    f1 = make_scorer(f1_score, pos_label = "Bad")
    
    def objective(space):        
        global best_score
        model = modelo(**space, random_state = 1)   
        cv =  StratifiedKFold(n_splits = 5, random_state = 1)
        score = -cross_val_score(model, X, y, cv = cv, scoring = f1, verbose = False).mean()
        if (score < best_score):
            best_score = score
        return score

    start = time.time()
    rstate = np.random.RandomState(1)
    best = fmin(objective, space = space,algo = tpe.suggest, max_evals = nevals, trials = Trials(), rstate = rstate)

    print("Hyperopt search took %.2f seconds" % ((time.time() - start)))
    print("Best score: %.4f " % (-best_score))
    print("Best space: ", space_eval(params, best))
    return(space_eval(params, best))

### KNN (KDTree Implementation)

In [12]:
tree = KDTree(X_train)

In [13]:
nearest_dist, nearest_ind = tree.query(X_test, k = 100)

In [14]:
pred_knn = [Counter([y_train.iloc[k] for k in x]).most_common(1)[0][0] for x in nearest_ind]

In [15]:
print('Accuracy Test :', f'{accuracy_score(y_test, pred_knn):.4f}', 
      '| F1 Test :', f'{f1_score(y_test, pred_knn, pos_label="Bad"):.4f}',
      '| Precision Test :', f'{precision_score(y_test, pred_knn, pos_label="Bad"):.4f}', 
      '| Recall Test :', f'{recall_score(y_test, pred_knn, pos_label="Bad"):.4f}', 
      '| H Test :', f'{H_score(y_test, pred_knn):.4f}')

Accuracy Test : 0.6710 | F1 Test : 0.6670 | Precision Test : 0.6752 | Recall Test : 0.6590 | H Test : 0.6690


### Gradient Boosted Trees

In [16]:
params = {'learning_rate':     hp.choice('learning_rate',[0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 
                                                          0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.50, 0.75, 1]), 
          'n_estimators':      hp.choice('n_estimators', range(1,400)),
          'max_depth':         hp.choice('max_depth',range(1,20)),
          'min_samples_split': hp.choice('min_samples_split',np.linspace(0.01, 1.0, 10, endpoint=True)),
          'min_samples_leaf':  hp.choice('min_samples_leaf',np.linspace(0.01, 0.5, 50, endpoint=True)), 
          'subsample':         hp.choice('subsample',[1]), 
          'max_features':      hp.choice('max_features',['sqrt'])}

best_score = 1
gbt_params = bayesian(params, X_train, y_train, GradientBoostingClassifier, 50)
pred_gbt = evaluate_model(GradientBoostingClassifier(**gbt_params, random_state=2), X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:41<00:00,  4.43s/trial, best loss: -0.6674859132976704]
Hyperopt search took 221.55 seconds
Best score: 0.6675 
Best space:  {'learning_rate': 0.005, 'max_depth': 17, 'max_features': 'sqrt', 'min_samples_leaf': 0.09999999999999999, 'min_samples_split': 0.89, 'n_estimators': 267, 'subsample': 1}
Accuracy Test : 0.6565 | F1 Test : 0.6721 | Precision Test : 0.6429 | Recall Test : 0.7040 | H Test : 0.6642
Accuracy Test : 0.6538 | F1 Test : 0.6686 | Precision Test : 0.6408 | Recall Test : 0.6988 | H Test : 0.6611


### Random Forest

In [17]:
params = {'bootstrap':         hp.choice('bootstrap',[True, False]),
          'max_depth':         hp.choice('max_depth', range(1, 20)),
          'max_features':      hp.choice('max_features',['auto', 'sqrt']),
          'min_samples_leaf':  hp.choice('min_samples_leaf',np.linspace(0.01, 0.5, 50, endpoint=True)), 
          'min_samples_split': hp.choice('min_samples_split',np.linspace(0.01, 1.0, 100, endpoint=True)), 
          'n_estimators':      hp.choice('n_estimators',range(1,400))}

best_score = 1
rf_params = bayesian(params, X_train, y_train, RandomForestClassifier, 50)
pred_rf = evaluate_model(RandomForestClassifier(**rf_params, random_state=2), X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:52<00:00,  5.86s/trial, best loss: -0.6644088088816696]
Hyperopt search took 293.22 seconds
Best score: 0.6644 
Best space:  {'bootstrap': True, 'max_depth': 11, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 'min_samples_split': 0.47000000000000003, 'n_estimators': 318}
Accuracy Test : 0.6580 | F1 Test : 0.6699 | Precision Test : 0.6474 | Recall Test : 0.6940 | H Test : 0.6639
Accuracy Test : 0.6545 | F1 Test : 0.6663 | Precision Test : 0.6440 | Recall Test : 0.6903 | H Test : 0.6604


### XGBoosting

In [18]:
params = {'learning_rate':    hp.choice('learning_rate',[0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 
                                                         0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75]), 
          'max_depth':        hp.choice('max_depth',range(1,20)),
          'min_child_weight': hp.choice('min_child_weight',np.linspace(0.01, 1.0, 100, endpoint=True)),
          'gamma':            hp.choice('gamma',np.linspace(0.01, 1.0, 100, endpoint=True)), 
          'colsample_bytree': hp.choice('colsample_bytree',np.linspace(0.0, 1, 101, endpoint=True)), 
          'n_estimators':     hp.choice('n_estimators', range(1,200))}

best_score = 1
xgb_params = bayesian(params, X_train, y_train, xgb.XGBClassifier, 50)
pred_xgb = evaluate_model(xgb.XGBClassifier(**xgb_params, random_state=2), X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [20:06<00:00, 24.13s/trial, best loss: -0.6666455251210933]
Hyperopt search took 1206.85 seconds
Best score: 0.6666 
Best space:  {'colsample_bytree': 0.75, 'gamma': 0.89, 'learning_rate': 0.0075, 'max_depth': 1, 'min_child_weight': 0.81, 'n_estimators': 128}
Accuracy Test : 0.6565 | F1 Test : 0.6746 | Precision Test : 0.6409 | Recall Test : 0.7120 | H Test : 0.6654
Accuracy Test : 0.6509 | F1 Test : 0.6672 | Precision Test : 0.6371 | Recall Test : 0.7004 | H Test : 0.6589


### Logistic Regresion

In [19]:
params = {"C":   hp.choice('C',[0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]),
          ""
          "tol": hp.choice('tol',[0.00001, 0.000025, 0.00005, 0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.025, 
                                  0.05, 0.1])}

best_score = 1
log_params = bayesian(params, X_train, y_train, LogisticRegression, 50)
pred_log = evaluate_model(LogisticRegression(**log_params, max_iter = 1000, random_state=2), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:48<00:00,  1.03trial/s, best loss: -0.664033619180916]
Hyperopt search took 48.76 seconds
Best score: 0.6640 
Best space:  {'C': 0.1, 'tol': 0.1}
Accuracy Test : 0.6660 | F1 Test : 0.6599 | Precision Test : 0.6722 | Recall Test : 0.6480 | H Test : 0.6629
Accuracy Test : 0.6719 | F1 Test : 0.6706 | Precision Test : 0.6729 | Recall Test : 0.6683 | H Test : 0.6712


### Decision Trees

In [20]:
params = {"max_depth":        hp.choice('max_depth', range(1, 50)),
          "max_features":     hp.choice('max_features', range(1, 50)),
          "min_samples_leaf": hp.choice('min_samples_leaf', range(1, 200)),
          "criterion":        hp.choice('criterion', ["gini", "entropy"])}

best_score = 1
tree_params = bayesian(params, X_train, y_train, DecisionTreeClassifier, 50)
pred_tree = evaluate_model(DecisionTreeClassifier(**tree_params, random_state=2), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.15trial/s, best loss: -0.678783872008715]
Hyperopt search took 23.37 seconds
Best score: 0.6788 
Best space:  {'criterion': 'entropy', 'max_depth': 3, 'max_features': 35, 'min_samples_leaf': 135}
Accuracy Test : 0.6490 | F1 Test : 0.6437 | Precision Test : 0.6536 | Recall Test : 0.6340 | H Test : 0.6463
Accuracy Test : 0.6505 | F1 Test : 0.6471 | Precision Test : 0.6531 | Recall Test : 0.6413 | H Test : 0.6488


### SVM (poly)

In [21]:
params = {"degree": hp.choice('degree', [2, 3, 4]),
          "kernel": hp.choice('kernel', ['poly']), 
          "C":      hp.choice('C', [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 
                                    0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75])}
best_score = 1
svm_params = bayesian(params, X_train, y_train, SVC, 10)
pred_svm = evaluate_model(SVC(**svm_params, random_state=2, probability = True), X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:53<00:00, 29.35s/trial, best loss: -0.6747928393340464]
Hyperopt search took 293.52 seconds
Best score: 0.6748 
Best space:  {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
Accuracy Test : 0.6670 | F1 Test : 0.6804 | Precision Test : 0.6541 | Recall Test : 0.7090 | H Test : 0.6736
Accuracy Test : 0.6705 | F1 Test : 0.6875 | Precision Test : 0.6535 | Recall Test : 0.7251 | H Test : 0.6789


### SVM (rbf)

In [22]:
params = {'C':      hp.choice('C', [1, 2, 5, 10, 15, 20]), 
          'gamma':  hp.choice('gamma', [0.0001, 0.001, 0.01, 0.1]),
          'kernel': hp.choice('kernel', ['rbf'])}

best_score = 1
svm_params_2 = bayesian(params, X_train, y_train, SVC, 10)
pred_svm_2 = evaluate_model(SVC(**svm_params_2, random_state=2, probability = True), X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:24<00:00, 38.44s/trial, best loss: -0.6720928102318984]
Hyperopt search took 384.40 seconds
Best score: 0.6721 
Best space:  {'C': 2, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy Test : 0.6715 | F1 Test : 0.6759 | Precision Test : 0.6670 | Recall Test : 0.6850 | H Test : 0.6737
Accuracy Test : 0.6695 | F1 Test : 0.6772 | Precision Test : 0.6615 | Recall Test : 0.6936 | H Test : 0.6733


### CatBoost

In [23]:
cat_features = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37]
params = {'iterations':        hp.choice('iterations', range(100, 4000, 25)), 
          'learning_rate':     hp.choice('learning_rate', [0.001, 0.0025, 0.0075, 0.01, 0.025, 0.05, 0.1]),
          'l2_leaf_reg':       hp.choice('l2_leaf_reg', range(1, 10)), 
          'cat_features':      hp.choice('cat_features', [cat_features]), 
          'verbose':           hp.choice('verbose', [False])}

best_score = 1
cat_params = bayesian(params, X_train, y_train, CatBoostClassifier, 10)
pred_cat = evaluate_model(CatBoostClassifier(**cat_params, random_state=2), X_train, X_test, y_train, y_test)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [27:12<00:00, 163.27s/trial, best loss: -0.6683009183923718]
Hyperopt search took 1632.82 seconds
Best score: 0.6683 
Best space:  {'cat_features': (5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37), 'iterations': 1050, 'l2_leaf_reg': 2, 'learning_rate': 0.0075, 'verbose': False}
Accuracy Test : 0.6735 | F1 Test : 0.6727 | Precision Test : 0.6744 | Recall Test : 0.6710 | H Test : 0.6731
Accuracy Test : 0.6991 | F1 Test : 0.7024 | Precision Test : 0.6945 | Recall Test : 0.7104 | H Test : 0.7007


### Best Model (SVM)

In [24]:
svm_params = {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
pred_svm = evaluate_model(SVC(**svm_params, random_state=2), X_train, X_test, y_train, y_test)

Accuracy Test : 0.6670 | F1 Test : 0.6804 | Precision Test : 0.6541 | Recall Test : 0.7090 | H Test : 0.6736
Accuracy Test : 0.6705 | F1 Test : 0.6875 | Precision Test : 0.6535 | Recall Test : 0.7251 | H Test : 0.6789


In [25]:
pd.crosstab(pred_svm, y_test)

Category,Bad,Good
row_0,,
Bad,709,375
Good,291,625


### Save Models

In [26]:
models = (tree,
          LogisticRegression(**log_params, max_iter = 1000).fit(X_train, y_train),
          SVC(**svm_params, probability = True).fit(X_train, y_train),
          SVC(**svm_params_2, probability = True).fit(X_train, y_train),
          DecisionTreeClassifier(**tree_params).fit(X_train, y_train),
          RandomForestClassifier(**rf_params).fit(X_train, y_train),
          GradientBoostingClassifier(**gbt_params).fit(X_train, y_train),
          xgb.XGBClassifier(**xgb_params).fit(X_train, y_train),
          CatBoostClassifier(**cat_params).fit(X_train, y_train)
)

In [27]:
pickle.dump(models, open('./sav/model_f1.sav', 'wb'))